<a href="https://colab.research.google.com/github/cmoweg/moning/blob/master/Lab_10_3_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'couda':
  torch.cuda.manual_seed_all(777)

In [0]:
#parameters 나중에 넣어도 관계없다
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [0]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/', # 따라써야 추가로 불필요한 다운로드를 하지 않는다.
                          train = True,
                          transform = transforms.ToTensor(), # Tensor value로 전환
                          download=True # 직접 데이터 다운로드해서 사용하겠다.
                          )
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train = False, # False, test data
                          transform = transforms.ToTensor(), 
                          download=True)

In [0]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle=True,
                                          drop_last=True)


In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc = nn.Linear(7*7*64, 10, bias = True)# channel이 64에 7 by 7 output
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)

    out = out.view(out.size(0), -1) # batch_size만큼 펼치고, 나머지는 한줄...???
    out = self.fc(out)
    return out

In [0]:
model = CNN().to(device)

In [0]:
# loss function
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # optimizer를 사용할 때 파라미터를 넣어줘야 한다.

In [17]:
  # training
  total_batch = len(data_loader) # 전체 batch 크기

  for epoch in range(training_epochs):
    avg_cost = 0 # loss를 담을 것.

    for X, Y in data_loader:
      X = X.to(device)
      Y = Y.to(device)

      optimizer.zero_grad() # 중요. 없으면 학습 불가
      hypothesis = model(X)

      cost = criterion(hypothesis, Y)
      cost.backward()
      optimizer.step()

      avg_cost += cost / total_batch # cost를 쌓는다.

    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
  print('Learning Finished!')


[Epoch:1] cost = 0.22400668263435364
[Epoch:2] cost = 0.06218694895505905
[Epoch:3] cost = 0.04490301385521889
[Epoch:4] cost = 0.035570982843637466
[Epoch:5] cost = 0.029045002534985542
[Epoch:6] cost = 0.024852784350514412
[Epoch:7] cost = 0.020718909800052643
[Epoch:8] cost = 0.018198281526565552
[Epoch:9] cost = 0.015304670669138432
[Epoch:10] cost = 0.012417933903634548
[Epoch:11] cost = 0.010542315430939198
[Epoch:12] cost = 0.009918601252138615
[Epoch:13] cost = 0.008947704918682575
[Epoch:14] cost = 0.007122100796550512
[Epoch:15] cost = 0.005885852966457605
Learning Finished!


In [18]:
# test model
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean() # 평균
  print('Accuracy:', accuracy.item())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9883000254631042
